In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import dotenv
import os

percorso_credenziali = '../credenziali/credenziali_cartella.env'
dotenv.load_dotenv(override = True, dotenv_path = percorso_credenziali)

cartella = os.getenv('Download')

# Load catalog
catalog_path = f'{cartella}\\capstonehp.csv'
catalog = pd.read_csv(catalog_path)

# Normalize Paese values to English names if necessary
catalog['Paese'] = catalog['Paese'].astype(str).str.strip()

# Prepare countries and monthly order ranges
countries = {
    'Germany': (200, 400),
    'France': (150, 300),
    'Spain': (100, 200)
}

# Months to simulate: September and October 2025
months = [
    ('2025-09-01', '2025-09-30'),
    ('2025-10-01', '2025-10-31')
]

# Utility functions for selection logic
def pick_by_weight(df, weights):
    return df.sample(weights=weights, replace=True).iloc[0]

def date_random_between(start_str, end_str):
    start = datetime.fromisoformat(start_str)
    end = datetime.fromisoformat(end_str)
    delta = (end - start).days + 1
    rand_day = random.randrange(delta)
    # random time within the day
    rand_time = timedelta(seconds=random.randint(0, 86399))
    return (start + timedelta(days=rand_day) + rand_time).strftime("%Y-%m-%d %H:%M:%S")

# Scoring helpers for each country
def score_germany(row, ambito):
    price = row['Prezzo']
    category = row['Categoria'].lower()
    title = str(row['Titolo']).lower()
    # target price mid-range
    target = 1000 if category=='laptops' else 1200 if category=='desktops' else 200
    price_score = -abs(np.log1p(price) - np.log1p(target))
    # prefer technical titles (digits, model-like words)
    tech_keywords = ['pro','work','station','workstation','book','hp','omen','g','series','model','16','14','17','i7','ryzen']
    tech_bonus = sum([1 for k in tech_keywords if k in title])
    # ambito match bonus: Germaners picky about correct ambito (Home/Work)
    ambito_bonus = 0.5 if row['Ambito'].lower()==ambito.lower() else 0.0
    return price_score + 0.2*tech_bonus + ambito_bonus

def score_france(row, ambito):
    price = row['Prezzo']
    category = row['Categoria'].lower()
    title = str(row['Titolo']).lower()
    # target price leans higher (prestige)
    target = 1200 if category=='laptops' else 1400 if category=='desktops' else 250
    price_score = -abs(np.log1p(price) - np.log1p(target))
    stylish_keywords = ['vision','élite','elite','prestige','air','elegant','spectre','envy','studio','zenbook','craft']
    style_bonus = sum([1 for k in stylish_keywords if k in title])
    ambito_bonus = 0.4 if row['Ambito'].lower()==ambito.lower() else 0.0
    return price_score + 0.3*style_bonus + ambito_bonus

def score_spain(row, ambito):
    price = row['Prezzo']
    category = row['Categoria'].lower()
    title = str(row['Titolo']).lower()
    # target price lower (value)
    target = 800 if category=='laptops' else 900 if category=='desktops' else 150
    price_score = -abs(np.log1p(price) - np.log1p(target))
    catchy_keywords = ['speed','go','power','flex','smart','life','plus','max','pro','ultra']
    catchy_bonus = sum([1 for k in catchy_keywords if k in title])
    ambito_bonus = 0.3 if row['Ambito'].lower()==ambito.lower() else 0.0
    return price_score + 0.25*catchy_bonus + ambito_bonus

# Create orders
orders = []
order_id_counter = 1000000

for country, (min_ord, max_ord) in countries.items():
    # filter catalog for country products; if none, fallback to whole catalog
    country_catalog = catalog[catalog['Paese'].str.lower()==country.lower()]
    if country_catalog.empty:
        country_catalog = catalog.copy()
    # Decide monthly counts for Sept and Oct
    for month_start, month_end in months:
        n_orders = random.randint(min_ord, max_ord)
        for i in range(n_orders):
            order_id = order_id_counter
            order_id_counter += 1
            # Random date in month
            date = date_random_between(month_start, month_end)
            # Decide ambito (Home/Work) probabilities tuned per country
            if country=='Germany':
                ambito = np.random.choice(['Home','Work'], p=[0.4, 0.6])
            elif country=='France':
                ambito = np.random.choice(['Home','Work'], p=[0.55, 0.45])
            else: # Spain
                ambito = np.random.choice(['Home','Work'], p=[0.65, 0.35])
            # Decide if this order is an exception (random small chance)
            is_exception = random.random() < 0.08  # 8% exceptions
            # Decide if multi-item order
            is_multi = random.random() < 0.07  # 7% multi-item
            # Decide if quantities >1 in this order
            has_multi_qty = random.random() < 0.12  # 12% have qty>1 for at least one line
            # Number of lines in order
            lines = random.randint(2,4) if is_multi else 1
            # For each line, choose product according to scoring or exception random
            chosen_skus = []
            for line in range(lines):
                # scoring
                pool = country_catalog.copy()
                # For exception: occasionally pick opposite ambito or random product indifferent to score
                if is_exception and random.random() < 0.6:
                    # pick random from full pool
                    prod = pool.sample(1).iloc[0]
                else:
                    # compute scores per country
                    if country=='Germany':
                        pool['score'] = pool.apply(lambda r: score_germany(r, ambito), axis=1)
                    elif country=='France':
                        pool['score'] = pool.apply(lambda r: score_france(r, ambito), axis=1)
                    else:
                        pool['score'] = pool.apply(lambda r: score_spain(r, ambito), axis=1)
                    # convert scores to probabilities (softmax-ish)
                    scores = pool['score'].values
                    # numeric stability
                    exps = np.exp((scores - np.max(scores))/1.0)
                    probs = exps / exps.sum()
                    prod = pool.sample(1, weights=probs).iloc[0]
                sku = prod['Sku']
                # ensure we can have different skus per order line; if duplicate happen allow but try to vary
                chosen_skus.append(prod)
            # Create order lines rows
            # Determine quantities
            for idx, prod in enumerate(chosen_skus):
                sku = prod['Sku']
                # quantity rules: monitors often bought multiple; corporate orders may buy multiple desktops for offices
                category = prod['Categoria'].lower()
                # base qty
                qty = 1
                if has_multi_qty:
                    if category=='monitors':
                        qty = random.choices([2,3,4], weights=[0.6,0.3,0.1])[0]
                    elif country=='Germany' and ambito=='Work' and category=='desktops':
                        qty = random.choices([2,3,4,5], weights=[0.6,0.25,0.1,0.05])[0]
                    else:
                        qty = random.choices([2,3], weights=[0.8,0.2])[0]
                # Small chance of bulk corporate order for France/Spain
                if random.random() < 0.01:
                    qty += random.randint(5,20)
                orders.append({
                    'Paese': country,
                    'Data': date,
                    'ID_Ordine': order_id,
                    'Sku': sku,
                    'Quantità': qty
                })

# Convert to DataFrame and save CSV
orders_df = pd.DataFrame(orders)

# Sort for readability
orders_df['Data_dt'] = pd.to_datetime(orders_df['Data'])
orders_df = orders_df.sort_values(['Paese','Data_dt','ID_Ordine']).drop(columns=['Data_dt'])

orders_df =orders_df.dropna(subset=['Sku'])
orders_df = orders_df[orders_df['Sku'].str.strip() != ""]

out_path = f'{cartella}\\simulazione_vendite.csv'
orders_df.to_csv(out_path, index=False)

# Display basic summary
summary = orders_df.groupby('Paese').size().reset_index(name='Num_Ordini_Righe')
summary['Num_Ordini_Righe'] = summary['Num_Ordini_Righe'].astype(int)